In [1]:
import sys
import json
import numpy as np
params = {}
default = {'training_folder': '../../scenarios/houston/', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../../scenarios/houston/', 'params': {}}


In [2]:
def set_index(df):
    if 'index' in df.columns:
        df = df.set_index('index')
    else:
        df.index.name='index'
    return df

In [3]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
num_cores = nb.config.NUMBA_NUM_THREADS
print('num cores:',num_cores)

io_engine= 'pyogrio' if on_lambda else 'pyogrio' #or fiona

PyTables is not installed. No support for HDF output.
num cores: 8


In [4]:
base_folder = argv['training_folder']
input_folder = os.path.join(base_folder,'inputs/')
pt_folder  = os.path.join(input_folder,'pt/')
road_folder = os.path.join(input_folder,'road/')
od_folder =  os.path.join(input_folder,'od/')
params_folder = input_folder if on_lambda else '../../inputs/'

output_folder = os.path.join(base_folder,'outputs/')
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    
model_folder = os.path.join(input_folder, 'model/')

In [5]:
var = excel.read_var(file=os.path.join(params_folder,'parameters.xlsx'), scenario='base', return_ancestry=False)
if 'params' in argv.keys():
    var.update(pd.DataFrame.from_dict(argv['params'], orient="index").stack())

# inputs

In [6]:
sm = stepmodel.read_zippedpickles(model_folder + 'network')

zones: 100%|██████████| 25/25 [00:01<00:00, 20.30it/s]              


# pathfinder

In [7]:

sm.step_pt_pathfinder(
    broken_routes=False,
    broken_modes=False,
    keep_pathfinder=True,
    mode_column='route_type',
    route_column='route_id',
    speedup=True,
    walk_on_road=True,
    path_analysis=False,
    od_set= None,
    engine='b', # b is faster!
)


start publicpathfinder
path_analysis


In [8]:

sm.step_road_pathfinder(method='aon', access_time='time',path_analysis=False)

self.volumes does not exist. od generated with self.zones


In [9]:
sm.to_zippedpickles(model_folder +'los', omitted_attributes=['publicpathfinder'])

car_los: 100%|██████████| 28/28 [00:10<00:00,  2.74it/s]      
